In [29]:
from datasets import load_dataset
from transformers import BertTokenizer
from datasets import DatasetDict
import numpy as np
# Instantiate a tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [3]:
train_dataset = load_dataset("csv", data_files="/home/ghan/caml-mimic-fixed-/mimicdata/mimic3/train_50.csv")
eval_dataset = load_dataset("csv", data_files="/home/ghan/caml-mimic-fixed-/mimicdata/mimic3/dev_50.csv")
predict_dataset = load_dataset("csv", data_files="/home/ghan/caml-mimic-fixed-/mimicdata/mimic3/test_50.csv")

Using custom data configuration default-2828371e2f74ebe3
Found cached dataset csv (/home/ghan/.cache/huggingface/datasets/csv/default-2828371e2f74ebe3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-6249d4195201014c
Found cached dataset csv (/home/ghan/.cache/huggingface/datasets/csv/default-6249d4195201014c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-b097882698f56793
Found cached dataset csv (/home/ghan/.cache/huggingface/datasets/csv/default-b097882698f56793/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
train_dataset['train']

Dataset({
    features: ['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS', 'length'],
    num_rows: 8066
})

In [12]:
def encode(examples):
    encoded = tokenizer(examples['TEXT'])
    examples['length'] = len(encoded['input_ids'])
    return examples

In [18]:
predict_dataset = predict_dataset.map(encode)

  0%|          | 0/1729 [00:00<?, ?ex/s]

In [19]:
eval_dataset = eval_dataset.map(encode)

  0%|          | 0/1573 [00:00<?, ?ex/s]

In [20]:
train_dataset = train_dataset.map(encode)

  0%|          | 0/8066 [00:00<?, ?ex/s]

In [34]:
train_dataset['train']['length'][-5000]

1626

In [26]:
eval_dataset['train']['length']

1573

In [23]:
lengths=train_dataset['train']['length']+predict_dataset['train']['length']+eval_dataset['train']['length']

In [27]:
len(lengths)

11368

In [30]:
quartiles = np.percentile(lengths, [25, 50, 75])

In [31]:
quartiles

array([1432.  , 2022.  , 2741.25])

In [37]:
def add_length_feature(examples):

    length=examples['length']
    if length <= quartiles[0]:
        length_feature=1
    elif length <= quartiles[1]:
        length_feature=2
    elif length <= quartiles[2]:
        length_feature=3
    else:
        length_feature=4
    examples['length_feature'] = length_feature
    return examples

train_dataset = train_dataset['train'].map(add_length_feature)
predict_dataset = predict_dataset['train'].map(add_length_feature)
eval_dataset = eval_dataset['train'].map(add_length_feature)

  0%|          | 0/8066 [00:00<?, ?ex/s]

  0%|          | 0/1729 [00:00<?, ?ex/s]

  0%|          | 0/1573 [00:00<?, ?ex/s]

In [41]:
train_dataset

Dataset({
    features: ['SUBJECT_ID', 'HADM_ID', 'TEXT', 'LABELS', 'length', 'length_feature'],
    num_rows: 8066
})

In [ ]:
train_df = train_dataset.to_pandas()
eval_df=eval_dataset.to_pandas()
predict_df=predict_dataset.to_pandas()

In [ ]:
train_df.to_json("/home/ghan/datasets/mimic_train50.json", orient="records", lines=True)
eval_df.to_json("/home/ghan/datasets/mimic_dev50.json", orient="records", lines=True)
predict_df.to_json("/home/ghan/datasets/mimic_test50.json", orient="records", lines=True)